In [19]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
from tweepy import Stream                   
from tweepy.streaming import StreamListener
import json
import csv
import os
from os import path
import got3

In [20]:
consumer_key = "xxxxxxxxxx"
consumer_secret = "xxxxxxxxxx"
token_key = "xxxxxxxxxx"
token_secret = "xxxxxxxxxx"

In [21]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(token_key, token_secret)
api = tweepy.API(auth)

In [23]:
tweetCriteria = got3.manager.TweetCriteria().setQuerySearch('AMZN').setSince("2018-10-01").setUntil("2019-10-15")
tweet = got3.manager.TweetManager.getTweets(tweetCriteria)
print (len(tweet))

107762


In [24]:
def write_to_csv(tweet):
    #fields = ["timestamp", "tweet", "user", "verified", "url"]
    fields = ["Date", "Tweet", "PermLink", "retweets", "Favourites", "url", "Geo"]
    filename = "C:\k.v.singh\sjsu\Fall 2019\CMPE257\TwitterData\Amazon.csv"
    fileState = path.isfile(filename)
    with open(filename,'a') as fd:        
        writer = csv.writer(fd)
        if(not fileState):
            writer.writerow(fields)
        else:
            #print(tweet)
            writer.writerow([tweet.date, tweet.text.encode("utf-8"), tweet.permalink.encode("utf-8"),tweet.retweets, tweet.favorites,tweet.urls.encode("utf-8"),tweet.geo.encode("utf-8")])
        #fd.write(timestamp, tweet, user, verified, url)       

In [25]:
for idx in range(len(tweet)):
    write_to_csv(tweet[idx])

In [26]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) | (\w+:\/\/\S+)", " ", tweet).split())

In [27]:
def get_tweet_sentiment(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'

In [64]:
import pandas as pd
 
tweets_data_path = "C:\k.v.singh\sjsu\Fall 2019\CMPE257\TwitterData\Amazon.csv"
pdData = pd.read_csv(tweets_data_path, delimiter=",")
for row in pdData['Tweet']:    
    try:
        parsedTweet = {}
        #parsedTweet['date'] = row['Date']
        parsedTweet['text'] = row
        parsedTweet['sentiment'] = get_tweet_sentiment(row)
        tweets_data.append(parsedTweet)
    except tweepy.TweepError as e:
        print("Error : " + str(e))


In [60]:
#ptweets = [tweet for tweet in tweets_data if tweets_data['sentiment'] == 'positive']
ptweets = [tweet for tweet in tweets_data if tweet['sentiment'] == 'positive']
print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets_data)))
ntweets = [tweet for tweet in tweets_data if tweet['sentiment'] == 'negative']
print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets_data)))
print("Neutral tweets percentage: {} %".format(100*(len(tweets_data) - len(ntweets) - len(ptweets))/len(tweets_data)))

Positive tweets percentage: 35.32354005623556 %
Negative tweets percentage: 10.926030753240969 %
Neutral tweets percentage: 53.75042919052348 %
